# 利用狄利克雷边界条件解决拉普拉斯问题

### 背景[1]

In this tutorial, we will solve a simple Laplace problem inside the unit sphere with Dirichlet boundary conditions. Let $\Omega$ be the unit sphere with boundary $\Gamma$. Let $\nu$ be the outward pointing normal on $\Gamma$. The PDE and boundary conditions are given by

\begin{align}
\Delta u &= 0&&\text{in }\Omega,\\
u &= f&&\text{on }\Gamma.
\end{align}

The boundary data is a source $\hat{u}$ located at the point $(0.9,0,0)$.
$$
\hat{u}(\mathbf x)=\frac{1}{4\pi\sqrt{(x-0.9)^2+y^2+z^2}}.
$$

For this example, we will use a direct integral equation of the first kind. Let
$$
g(\mathbf x,\mathbf y) = \frac{1}{4\pi |\mathbf x-\mathbf y|}
$$
be the Green's function for Laplace in three dimensions. From Green's representation theorem, it follows that every harmonic function $u$ in $\Omega$ satisfies

$$
u(\mathbf x) = \int_{\Gamma} g(\mathbf x,\mathbf y)\frac{\partial u(\mathbf y)}{\partial \nu(\mathbf{y})}\mathrm{d}\mathbf y-\int_{\Gamma}\frac{\partial g(\mathbf x,\mathbf y)}{\partial \nu(\mathbf{y})}u(\mathbf y)\mathrm{d}\mathbf y,~\mathbf x\in\Omega\setminus\Gamma
$$

or equivalantly

$$
u(\mathbf x) = \left[\mathcal{V}\frac{\partial u(\mathbf y)}{\partial \nu(\mathbf{y})}\right] (\mathbf{x}) - \left[\mathcal{K}u\right] (\mathbf{x}),~\mathbf x\in\Omega\setminus\Gamma,
$$

where $\mathcal{V}$ and $\mathcal{K}$ are the <a href='https://bempp.com/2017/07/11/available_operators/'>single and double layer potential operators</a>.

Taking the limit $\mathbf x\rightarrow \Gamma$ we obtain the boundary integral equation

$$
\left[\mathsf{V}\frac{\partial u}{\partial n}\right] (\mathbf x)=\left[(\tfrac12\mathsf{Id}+\mathsf{K})u\right] (\mathbf x),~\mathbf x\in\Gamma.
$$

Here, $\mathsf{V}$ and $\mathsf{K}$ are the <a href='https://bempp.com/2017/07/11/available_operators/'>single and double layer boundary operators</a>.

[1]背景及部分操作内容参考自 bempp tutorials


### 利用Dirichlet 边界条件计算离子阱电极外电势分布

步骤：
* 利用Gmsh制作电极排布，并划分2D网格，生成.msh文件。
  - 利用.geo脚本生成电极，注意需要用 physcial surface 标记不同电极的域 
  - 脚本里设置的电极尺寸在下面的计算中将会用到
    
* 导入msh文件，执行计算


### 计算电势的空间分布情况

利用Bempp解决问题. 首先导入Bempp and NumPy包.

In [ ]:
import bempp.api
import numpy as np
import meshio
import pygmsh as pg

导入生成的.msh文件

In [ ]:
#grid = bempp.api.shapes.sphere(h=0.1)
grid = bempp.api.import_grid('Blade_new.msh')

We now define the <a href='https://bempp.com/2017/07/11/function-spaces/'>spaces</a>. For this example we will use two spaces: the space of continuous, piecewise linear functions; and the space of piecewise constant functions. The space of piecewise constant functions has the right smoothness for the unknown Neumann data. We will use continuous, piecewise linear functions to represent the known Dirichlet data.

狄利克雷是第一类边界条件，诺依曼是第二类边界条件，计算时需要先为两类边界条件划分空间

In [ ]:
dp0_space = bempp.api.function_space(grid, "DP", 0)
p1_space = bempp.api.function_space(grid, "P", 1)

We can now define the <a href='https://bempp.com/2017/07/11/operators/'>operators</a>. We need the identity, single layer, and double layer boundary operator.

定义单层和双层的边界算符，即 $\mathcal{V}$ 和 $\mathcal{K}$

In [ ]:
identity = bempp.api.operators.boundary.sparse.identity(
    p1_space, p1_space, dp0_space)
dlp = bempp.api.operators.boundary.laplace.double_layer(
    p1_space, p1_space, dp0_space)
slp = bempp.api.operators.boundary.laplace.single_layer(
    dp0_space, p1_space, dp0_space)

We now define the <a href='https://bempp.com/2017/07/11/grid-functions/'>GridFunction object</a> on the sphere grid that represents the Dirichlet data.

接下来确定狄利克雷边界上的数值，此处即用到了生成.msh文件时的surface的分组，在我的文件中，surface[1] group了一对相对的电极的表面，将其作为RF的一对电极，surface[2] group了另外的一对电极，将其作为DC的电极表面。

In [ ]:
i =0
def dirichlet_data(x, n, domain_index, result):
#     print(domain_index)
#     result[0] = 1./(4 * np.pi * ((x[0] - 1.5)**2 + x[1]**2 + x[2]**2)**(0.5))
    global i
    if domain_index in [1]:      # 在不同区域设置相应的电压
        result[0] = 300.0 * np.sin(np.pi / 6 * 9)
        i+=1
#         print(domain_index)
    elif domain_index in [2]:
        result[0] = 0.0
    else:
        result[0] = 0.0
#        print('other')
#        print(domain_index)
    
dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

We next assemble the right-hand side of the boundary integral equation, given by $$(-\tfrac12\mathsf{Id}+\mathsf{K})u.$$

In [ ]:
%time rhs = (-.5 * identity + dlp) * dirichlet_fun  # solve exterior
# rhs = (.5 * identity + dlp) * dirichlet_fun  # solve interior

We now <a href='https://bempp.com/2017/07/12/solving-linear-systems/'>solve the linear system</a> using a conjugate gradient (CG) method.

In [ ]:
%time neumann_fun, info = bempp.api.linalg.cg(slp, rhs, tol=1E-3)

We now want to provide a simple plot of the solution in the $(x,y)$ plane for $z=0$. We first define points at which to plot the solution.


In [ ]:
# plot_grid 计算电势时包含的范围，尽量包含所有想要求解的边界条件，此时即要注意在生成电极时使用的尺寸。 单位:米
# n_grid_points 将plot_grid 分为多少份，越大精细度越高，但同时计算时间越长
# np.mgrid() 如果是两维，说明计算电势时只计算了一个平面，此时的边界条件是线
# 如果是三维， 则计算空间电势，此时的边界条件是面，计算时间增长

n_grid_points = 1000
plot_grid = np.mgrid[-10000:10000:n_grid_points*1j, -10000:10000:n_grid_points*1j]
points = np.vstack((plot_grid[0].ravel(),
                    plot_grid[1].ravel(),
                    np.zeros(plot_grid[0].size)))

The variable `points` now contains in its columns the coordinates of the evaluation points. We can now use Green's representation theorem to evaluate the solution on these points. Note in particular the last line of the following code. It is a direct implementation of Green's representation theorem.

In [ ]:
%%time
slp_pot = bempp.api.operators.potential.laplace.single_layer(
    dp0_space, points)
dlp_pot = bempp.api.operators.potential.laplace.double_layer(
    p1_space, points)
u_evaluated = slp_pot * neumann_fun - dlp_pot * dirichlet_fun


### 画图分析空间的电势情况

分别作出Z = 0平面处的电势的热图、三维图和等势面图

In [ ]:
# The next command ensures that plots are shown within the IPython notebook
%matplotlib inline

import matplotlib
from matplotlib import pylab as plt
import seaborn as sns
# halfrange*2 代表了画图时画出的电势的范围
halfrange = 40


# Filter out solution values that are associated with points outside the unit circle.
u_evaluated = u_evaluated.reshape((n_grid_points,n_grid_points))

radius = np.sqrt(plot_grid[0]**2 + plot_grid[1]**2)
#u_evaluated[radius>1] = np.nan

# Plot the image

plt.figure(figsize=(10,6))
cmap = sns.diverging_palette(220, 20, n=7,as_cmap=True)
#data = u_evaluated
data = u_evaluated[n_grid_points/2-halfrange:n_grid_points/2+halfrange,n_grid_points/2-halfrange:n_grid_points/2+halfrange]

plt.imshow(data,aspect=1.0,cmap=cmap)
plt.title('Computed solution')
plt.grid()


# 3D图
from mpl_toolkits.mplot3d import Axes3D
X = np.arange(-halfrange,halfrange,1)
Y = np.arange(-halfrange,halfrange,1)
X, Y = np.meshgrid(X,Y)
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(X, Y, data, rstride=1, cstride=1, cmap='rainbow')
ax.set_title('potential-3D')
plt.show()

# 等势面图
plt.contourf(X, Y, data, 8, alpha=.75, cmap=plt.cm.hot)
plt.contour(X, Y, data, 8, colors='black', linewidth=.5)
plt.title('equipotential surface')
plt.show()

计算电势最低点和电势最高点的夹角

In [ ]:
#compute the angle of two axis
max_position = tuple(np.where(data == np.min(data)) - np.array(len(data)/2)) 
min_position = tuple(np.where(data == np.max(data)) - np.array(len(data)/2))
multi = max_position[0] * min_position[0] + max_position[1] * min_position[1]
molde = np.sqrt(max_position[0] ** 2 + max_position[1] **2) * np.sqrt(min_position[0] ** 2 + min_position[1] **2)
angle = np.arccos(multi / molde)

print angle / np.pi * 180  # unit: degree

画出X=0和Y=0处的电势线

In [ ]:
import seaborn as sns

data = u_evaluated.T[n_grid_points/2,n_grid_points/2-10:n_grid_points/2+10]
plt.plot(data,'ro')

# 多项式拟合得到的数据数据
x = np.arange(len(data))
opts = np.polyfit(x,data,4)
print(opts[1]/opts[2],opts) # 3阶与二阶的比值

In [ ]:
import seaborn as sns
data = u_evaluated.T[n_grid_points/2-10:n_grid_points/2+10,n_grid_points/2+10]
plt.plot(data,'ro')
x = np.arange(len(data))
opts = np.polyfit(x,data,4)
print(opts[1]/opts[2],opts) # 3阶与二阶的比值

### 计算RF电势旋转下的赝势能

生成三维网格重新计算电势

In [ ]:
%%time
import datetime

# 打印出计算这个cell需要的时间
print(datetime.datetime.now())

n_grid_points = 1000
L = 10000
dL = L/n_grid_points
plot_grid = np.mgrid[-L:L:n_grid_points*1j, -L:L:n_grid_points*1j,-L:L:n_grid_points*1j]
points = np.vstack((plot_grid[0].ravel(),
                    plot_grid[1].ravel(),
                    plot_grid[2].ravel()))
slp_pot = bempp.api.operators.potential.laplace.single_layer(
    dp0_space, points)
dlp_pot = bempp.api.operators.potential.laplace.double_layer(
    p1_space, points)
u_evaluated = slp_pot * neumann_fun - dlp_pot * dirichlet_fun

print(datetime.datetime.now())


利用电势求三个方向的梯度得出电场强度

In [ ]:
u3D =u_evaluated.reshape(((n_grid_points,n_grid_points,n_grid_points)))
Ex, Ey, Ez = np.gradient(u3D)

计算赝势能并画图

In [ ]:
#计算赝势能
fig, axarr = plt.subplots(1,3,figsize=(15,3.6))

from scipy.constants import m_n, m_p, m_u,e, electron_volt, pi
from scipy import ndimage

omega = 25e6*2*pi

# 赝势能公式
U_eff = e**2*(Ex**2+Ey**2+Ez**2)/(4*171*m_n*omega**2)/electron_volt /(dL*1e-6)**2   # L unit is um 

ax,ay = np.mgrid[-L:L:n_grid_points*1j, -L:L:n_grid_points*1j]
cmap = sns.diverging_palette(220, 20, n=7,as_cmap=True)


# dat= ndimage.zoom(U_eff[25], 3)
a = axarr[0].contourf(ax,ay,U_eff[25],cmap=cmap)
axarr[0].grid()
axarr[0].axis('equal')
fig.colorbar(a, ax=axarr[0], shrink=0.9, format='%1.1e')

a = axarr[1].contourf(ax,ay,U_eff[:,25,:],cmap=cmap)
axarr[1].grid()
axarr[1].axis('equal')
fig.colorbar(a, ax=axarr[1], shrink=0.9, format='%1.1e')

a = axarr[2].contourf(ax,ay,U_eff[:,:,25],cmap=cmap)
axarr[2].grid()
axarr[2].axis('equal')
fig.colorbar(a, ax=axarr[2], shrink=0.9, format='%1.1e')

# a = axarr[1].imshow(U_eff[:,25,:],cmap=cmap)
# axarr[1].grid()

# a = axarr[2].imshow(U_eff[:,:,25],cmap=cmap)
# axarr[2].grid()
# a.colorbar()
